## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(1) # It has 34299 rows

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1


What variable(s) are the target(s) for your model?
`IS_SUCCESSFUL`

What variable(s) are the feature(s) for your model?
`APPLICATION_TYPE`,`AFFILIATION	CLASSIFICATION`, `USE_CASE`, `ORGANIZATION`, `STATUS`, `INCOME_AMT`, `SPECIAL_CONSIDERATIONS`, `ASK_AMT`

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
app_df=application_df.copy().drop(columns=['EIN','NAME'])
app_df.head(1) # It has 34299 rows

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
app_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
show_app= app_df['APPLICATION_TYPE'].value_counts()
show_app= show_app[show_app < 529]
show_app=show_app.copy()

# Replace in dataframe
def update_app(value):
    if value in show_app: #['T9','T13','T12','T2','T25','T14','T29','T15','T17']:
        return 'OTHER'
    else:
        return value
app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].apply(update_app)
app_df['APPLICATION_TYPE'].value_counts()

# Check to make sure binning was successful
app_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
OTHER      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
# app_df['CLASSIFICATION'].value_counts()[app_df['CLASSIFICATION'].value_counts() > 1].sort_values(ascending=False)
show_class= app_df['CLASSIFICATION'].value_counts().copy()
show_class[show_class > 1]#.sort_values(ascending=False)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
show_class= show_class[show_class < 1883]
show_class=show_class.copy()

# Replace in dataframe
def update_app(value):
    if value in show_class:
        return 'OTHER'
    else:
        return value
app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].apply(update_app)
app_df['CLASSIFICATION'].value_counts()
    
# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
OTHER     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
Do_not_touch = ['STATUS','IS_SUCCESSFUL']
Do_touch = [
    'APPLICATION_TYPE',
    'AFFILIATION',
    'CLASSIFICATION',
    'USE_CASE',
    'ORGANIZATION',
    'INCOME_AMT',
    'SPECIAL_CONSIDERATIONS',
    'ASK_AMT'
]

# Apply pd.get_dummies() on the specified columns
dummies = pd.get_dummies(app_df[Do_touch])

# Concatenate the dummy variables with the original DataFrame
app_df= pd.concat([app_df, dummies], axis=1)
app_df= app_df.drop(columns=Do_touch)

In [10]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X=app_df.drop(columns='IS_SUCCESSFUL')
y=app_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
xs= scaler.fit(X_train)

# Scale the data
X_train_scaled = xs.transform(X_train)
X_test_scaled = xs.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=100, input_dim=41, activation='relu'))#tanh

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # Binary label prediction

In [13]:
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4200      
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 24501 (95.71 KB)
Trainable params: 24501 (95.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(
    loss = 'binary_crossentropy', # This loss function is needed for Binary label prediction
    optimizer='adam', # The algorith for gradient descent
    metrics=['accuracy'] # The metrics to improve the model
)

In [15]:
# Train the model
#  YOUR CODE GOES HERE
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5669 - accuracy: 0.7243
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5554 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7328
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5498 - accuracy: 0.7328
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5488 - accuracy: 0.7344
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7358
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5460 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5457 - accuracy: 0.7348
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5451 - accura

# After multiple tries the best I can find was 74.25

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5914 - accuracy: 0.7262 - 761ms/epoch - 3ms/step
Loss: 0.5913605690002441, Accuracy: 0.7261807322502136


In [17]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
import h5py
hdf5_filename = 'AlphabetSoupCharity_Optimization.h5'
nn.save(hdf5_filename)
print('Model saved to : ',hdf5_filename)

Model saved to :  AlphabetSoupCharity_Optimization.h5


c:\Users\pammo\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
